In [1]:
import pandas as pd
import numpy as np
import pickle
from faker import Faker
import df_helper as dfh

df_helper = dfh.df_helper()

# Initialize Faker
fake = Faker()

admission_types = [
    'URGENT', 'ELECTIVE', 'EW EMER.', 'DIRECT EMER.', 'EU OBSERVATION',
    'OBSERVATION ADMIT', 'DIRECT OBSERVATION',
    'AMBULATORY OBSERVATION', 'SURGICAL SAME DAY ADMISSION'
]
admission_locations = [
    'TRANSFER FROM HOSPITAL', 'TRANSFER FROM SKILLED NURSING FACILITY',
    'INTERNAL TRANSFER TO OR FROM PSYCH', 'PHYSICIAN REFERRAL',
    'EMERGENCY ROOM', 'PACU', 'PROCEDURE SITE',
    'WALK-IN/SELF REFERRAL', 'INFORMATION NOT AVAILABLE',
    'CLINIC REFERRAL'
]
insurance_types = [
    'Medicaid', 'Medicare', 'Other'
]
languages = [
    'ENGLISH', '?'
]
marital_statuses = [
    'SINGLE', 'MARRIED', 'DIVORCED', 'WIDOWED'
]
races = [
    'BLACK/CAPE VERDEAN', 'HISPANIC/LATINO - PUERTO RICAN', 'WHITE',
    'UNKNOWN', 'OTHER', 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - SALVADORAN', 'UNABLE TO OBTAIN',
    'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'HISPANIC/LATINO - CUBAN',
    'PATIENT DECLINED TO ANSWER', 'WHITE - BRAZILIAN',
    'HISPANIC OR LATINO'
]

drugs = [
    'Midodrine', 'Multivitamins W/minerals', 'Sodium Chloride',
    'Nephrocaps', 'Nicotine Polacrilex', 'OLANZapine', 'Ondansetron',
    'OxyCODONE (Immediate Release)'
]

diagnoses_types = [
    'Urinary tract infection, site not specified',
    'Acute respiratory failure',
    'Asthma, unspecified type, unspecified',
    'Solitary pulmonary nodule',
    'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
    'Dysphagia, oropharyngeal phase', 'Esophageal reflux', 'Hypoxemia',
    'Methicillin resistant pneumonia due to Staphylococcus aureus',
    'Attention deficit disorder with hyperactivity'
]

# List of sample medical supplies and equipment categories
inventory_categories = [
    'Medical Equipment', 'Pharmaceuticals', 'Surgical Tools',
    'Diagnostic Tools', 'PPE', 'Cleaning Supplies'
]

# List of sample item names
inventory_items = {
    'Medical Equipment': ['X-Ray Machine', 'Ultrasound Machine', 'MRI Scanner', 'ECG Monitor'],
    'Pharmaceuticals': ['Aspirin', 'Ibuprofen', 'Amoxicillin', 'Paracetamol'],
    'Surgical Tools': ['Scalpel', 'Forceps', 'Sutures', 'Hemostat'],
    'Diagnostic Tools': ['Thermometer', 'Blood Pressure Monitor', 'Pulse Oximeter'],
    'PPE': ['Face Masks', 'Gloves', 'Gowns', 'Face Shields'],
    'Cleaning Supplies': ['Disinfectant', 'Sanitizer', 'Alcohol Swabs', 'Wipes']
}

# Function to generate a random inventory item
# Counter for unique IDs
product_id_counter = 1
def generate_inventory_item():
    global product_id_counter
    category = np.random.choice(inventory_categories)
    item_name = np.random.choice(inventory_items[category])
    quantity = np.random.randint(10, 500)  # Random quantity of items
    price_per_unit = round(np.random.uniform(5, 5000), 2)  # Random price per unit
    expiry_date = fake.date_between(start_date='today', end_date='+3y')  # Random expiry date within 3 years

    item = {
        'product_id': product_id_counter,
        'category': category,
        'item_name': item_name,
        'quantity': quantity,
        'price_per_unit': price_per_unit,
        'total_value': round(quantity * price_per_unit, 2),
        'expiry_date': expiry_date
    }

    # Increment the product ID counter for the next item
    product_id_counter += 1

    return item

# Generate hospital inventory data
def generate_hospital_inventory(num_items):
    inventory = []
    for _ in range(num_items):
        inventory.append(generate_inventory_item())
    return pd.DataFrame(inventory)

def generate_patients(n):
    patients = []
    for i in range(n):
        gender = np.random.choice(['M', 'F'])
        patients.append({
            'patient_id': i,
            'name': fake.name_male() if gender == "M" else fake.name_female(),
            'age': np.random.randint(1, 90),
            'gender': gender,
            'insurance': np.random.choice(insurance_types),
            'language': np.random.choice(languages),
            'maritalStatus': np.random.choice(marital_statuses),
            'race': np.random.choice(races),
        })
    return pd.DataFrame(patients)

def generate_staff(n):
    staff = []
    for i in range(n):
        staff.append({
            'staff_id': i,
            'staff_name': fake.name(),
            'role': np.random.choice(['Physician', 'Nurse', 'Admin']),
            'shift_start': fake.time(),
            'shift_end': fake.time()
           
        })
    return pd.DataFrame(staff)

def generate_admissions(patients:pd.DataFrame):
    admissions = []
    for i in range(len(patients)):
        admissions.append({
            'adm_id': i,
            'patient_id': patients.iloc[i].patient_id,
            'admit_time': fake.date_time(),
            'discharge_time': None,
            'type': np.random.choice(admission_types),
            'location': np.random.choice(admission_locations),
            
        })
    return pd.DataFrame(admissions)

def generate_omr(admissions):
    table = []
    for i in range(len(admissions)):
        omr = {
            'omr_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'weight': np.random.randint(60, 120),
            ##'height': np.random.randint(150, 220)/ 10.0, #not used in modeling
            'bp_systolic': np.random.randint(80, 220),
            'bp_diastolic': np.random.randint(60, 120),
        }
        table.append(omr)
    return pd.DataFrame(table)

def generate_prescriptions(admissions):
    prescriptions = []
    for i in range(len(admissions)):
        prescriptions.append({
            'prescription_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'drug': np.random.choice(drugs),
        })
    return pd.DataFrame(prescriptions)

def generate_diagnoses(admissions:pd.DataFrame):
    diag = []
    for i in range(len(admissions)):
        diag.append({
            'diagnosis_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'diagnosis': np.random.choice(diagnoses_types),
        })
    return pd.DataFrame(diag)

def generate_rooms(n):
    rooms = []
    beds = []
    bed_taken = 0
    room_id = 0
    bed_id = 0
    for i in range(n):
        if i % 2 == 0:#create new room
            rooms.append({
                'room_id': room_id,
                'capacity': 1,
            })
            beds.append({
                "bed_id": bed_id,
                "room": room_id,
                "adm_id": None
            })
            room_id += 1
        else:
            room = np.random.choice(rooms)#add bed to existing room
            room["capacity"] += 1
            beds.append({
                "bed_id": bed_id,
                "room": room["room_id"],
                "adm_id": None
            })
        bed_id += 1

    return pd.DataFrame(rooms), pd.DataFrame(beds)

def assign_beds(admissions, beds):
    for i in range(len(admissions)):
        adm = admissions.iloc[i].adm_id
        bed = beds[beds.adm_id.isna()].sample(1).iloc[0,0]  #choose random empty bed
        beds.loc[bed, "adm_id"] = adm.astype(int)
        print(bed)
        
def match_doctors_nurses_to_patients(patients, staffs):
    doctors = staffs[staffs["role"] == "Physician"]
    nurses = staffs[staffs["role"] == "Nurse"]
    
    doctor_list = doctors[["staff_id", "staff_name"]].to_dict(orient="records")
    nurse_list = nurses[["staff_id", "staff_name"]].to_dict(orient="records")
    
    if len(doctor_list) == 0 or len(nurse_list) == 0:
        raise HTTPException(status_code=400, detail="Not enough doctors or nurses available for matching")
    
    # Shuffle the lists
    np.random.shuffle(doctor_list)
    np.random.shuffle(nurse_list)
    
    matched_assignments = []
    num_doctors = len(doctor_list)
    num_nurses = len(nurse_list)
    
    for i, patient in patients.iterrows():
        # Randomly pick a doctor and nurse
        assigned_doctor = np.random.choice(doctor_list)
        assigned_nurse = np.random.choice(nurse_list)
        
        matched_assignments.append({
            "patient_id": patient["patient_id"],
            "patient_name": patient["name"],
            "assigned_doctor": assigned_doctor["staff_name"],
            "assigned_nurse": assigned_nurse["staff_name"]
        })
    
    return matched_assignments


In [2]:
hospital_inventory = generate_hospital_inventory(50)
staffs = generate_staff(20)
patients = generate_patients(30)
admissions = generate_admissions(patients)
admissions.discharge_time = pd.to_datetime(admissions.discharge_time)
omr = generate_omr(admissions)
prescriptions = generate_prescriptions(admissions)
diagnoses = generate_diagnoses(admissions)
rooms, beds = generate_rooms(50)
assign_beds(admissions, beds)
match_doctors_nurses_to_patients(patients, staffs)

49
42
10
45
21
23
11
2
34
44
35
24
17
41
37
16
32
15
7
5
6
0
1
29
30
8
13
31
48
12


[{'patient_id': 0,
  'patient_name': 'Tyler Gentry',
  'assigned_doctor': 'Taylor Nash',
  'assigned_nurse': 'Kimberly Williams'},
 {'patient_id': 1,
  'patient_name': 'James Hawkins',
  'assigned_doctor': 'Taylor Nash',
  'assigned_nurse': 'Patrick Carter'},
 {'patient_id': 2,
  'patient_name': 'Jessica Cooper',
  'assigned_doctor': 'Christy Burns',
  'assigned_nurse': 'Patrick Carter'},
 {'patient_id': 3,
  'patient_name': 'Yolanda Daniel',
  'assigned_doctor': 'Trevor Boyer',
  'assigned_nurse': 'Kimberly Williams'},
 {'patient_id': 4,
  'patient_name': 'Ashley Hogan',
  'assigned_doctor': 'Trevor Boyer',
  'assigned_nurse': 'Kimberly Williams'},
 {'patient_id': 5,
  'patient_name': 'Ethan Love',
  'assigned_doctor': 'Emily Simmons',
  'assigned_nurse': 'Kimberly Williams'},
 {'patient_id': 6,
  'patient_name': 'Laura Johnson',
  'assigned_doctor': 'Taylor Nash',
  'assigned_nurse': 'Sarah Francis'},
 {'patient_id': 7,
  'patient_name': 'Michael Rosales',
  'assigned_doctor': 'Sergi

In [3]:
match_dnp = pd.DataFrame(match_doctors_nurses_to_patients(patients, staffs))

# Create a dictionary to store assigned patients by doctor and nurse
assigned_patients_dict = {}

# Iterate over the matched assignments to populate the dictionary
for _, row in match_dnp.iterrows():
    doctor_name = row['assigned_doctor']
    nurse_name = row['assigned_nurse']
    patient_name = row['patient_name']
    
    if doctor_name not in assigned_patients_dict:
        assigned_patients_dict[doctor_name] = []
    if nurse_name not in assigned_patients_dict:
        assigned_patients_dict[nurse_name] = []
        
    assigned_patients_dict[doctor_name].append(patient_name)
    assigned_patients_dict[nurse_name].append(patient_name)

# Update the 'staffs' DataFrame with assigned patients
staffs['assigned_patients'] = staffs['staff_name'].map(assigned_patients_dict)
staffs.head()

,staff_id,staff_name,role,shift_start,shift_end,assigned_patients
0,0,Mary Elliott,Physician,12:35:27,19:05:24,"[Michael Rosales, Angela Barrett, Jacob Wiley]"
1,1,William Lopez,Admin,11:39:36,15:52:58,NaN
2,2,Christopher Rivera,Admin,15:47:13,18:34:40,NaN
3,3,Sergio Martinez,Physician,08:23:58,03:17:13,"[Yolanda Daniel, Ashley Hogan, Tony Alvarado]"
4,4,Emily Simmons,Physician,07:20:19,12:22:54,"[Jessica Cooper, Donna Davidson, Craig Nelson]"


In [4]:
hospital_inventory.head()

,product_id,category,item_name,quantity,price_per_unit,total_value,expiry_date
0,1,Surgical Tools,Sutures,403,4831.22,1946981.66,2025-11-22
1,2,Cleaning Supplies,Disinfectant,393,18.23,7164.39,2025-06-14
2,3,Surgical Tools,Scalpel,449,668.47,300143.03,2026-12-20
3,4,Diagnostic Tools,Pulse Oximeter,249,3901.64,971508.36,2026-10-13
4,5,Medical Equipment,ECG Monitor,54,3914.29,211371.66,2026-05-01


In [5]:
staffs.head()

,staff_id,staff_name,role,shift_start,shift_end,assigned_patients
0,0,Mary Elliott,Physician,12:35:27,19:05:24,"[Michael Rosales, Angela Barrett, Jacob Wiley]"
1,1,William Lopez,Admin,11:39:36,15:52:58,NaN
2,2,Christopher Rivera,Admin,15:47:13,18:34:40,NaN
3,3,Sergio Martinez,Physician,08:23:58,03:17:13,"[Yolanda Daniel, Ashley Hogan, Tony Alvarado]"
4,4,Emily Simmons,Physician,07:20:19,12:22:54,"[Jessica Cooper, Donna Davidson, Craig Nelson]"


In [6]:
patients.head()

,patient_id,name,age,gender,insurance,language,maritalStatus,race
0,0,Tyler Gentry,11,M,Other,ENGLISH,SINGLE,BLACK/CAPE VERDEAN
1,1,James Hawkins,24,M,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN
2,2,Jessica Cooper,18,F,Medicare,?,WIDOWED,PORTUGUESE
3,3,Yolanda Daniel,63,F,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN
4,4,Ashley Hogan,59,F,Other,ENGLISH,DIVORCED,PORTUGUESE


In [7]:
admissions

,adm_id,patient_id,admit_time,discharge_time,type,location
0,0,0,1995-01-17 03:50:58.026009,NaT,SURGICAL SAME DAY ADMISSION,TRANSFER FROM SKILLED NURSING FACILITY
1,1,1,1981-09-02 21:21:32.536753,NaT,ELECTIVE,PROCEDURE SITE
2,2,2,1983-08-10 22:34:00.532622,NaT,ELECTIVE,EMERGENCY ROOM
3,3,3,1992-04-18 01:35:57.405371,NaT,DIRECT EMER.,TRANSFER FROM HOSPITAL
4,4,4,1974-03-16 16:57:02.552020,NaT,URGENT,INTERNAL TRANSFER TO OR FROM PSYCH
5,5,5,1996-02-25 23:42:09.339219,NaT,EW EMER.,PACU
6,6,6,2023-10-11 16:01:44.901767,NaT,OBSERVATION ADMIT,PROCEDURE SITE
7,7,7,2015-01-17 08:44:04.233517,NaT,ELECTIVE,CLINIC REFERRAL
8,8,8,1973-08-02 13:24:54.546436,NaT,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL
9,9,9,2022-03-28 06:40:38.048924,NaT,AMBULATORY OBSERVATION,TRANSFER FROM HOSPITAL


In [8]:
admissions.iloc[0,0]

0

In [9]:
omr.head()

,omr_id,adm_id,weight,bp_systolic,bp_diastolic
0,0,0,108,114,74
1,1,1,99,172,103
2,2,2,116,109,96
3,3,3,115,116,83
4,4,4,93,146,78


In [10]:
prescriptions.head()

,prescription_id,adm_id,drug
0,0,0,Sodium Chloride
1,1,1,Sodium Chloride
2,2,2,Nephrocaps
3,3,3,Nephrocaps
4,4,4,Nicotine Polacrilex


In [11]:
diagnoses.head()

,diagnosis_id,adm_id,diagnosis
0,0,0,Streptococcus infection in conditions classifi...
1,1,1,Hypoxemia
2,2,2,Attention deficit disorder with hyperactivity
3,3,3,"Dysphagia, oropharyngeal phase"
4,4,4,"Dysphagia, oropharyngeal phase"


In [12]:
rooms

,room_id,capacity
0,0,4
1,1,6
2,2,3
3,3,2
4,4,1
5,5,2
6,6,4
7,7,1
8,8,1
9,9,5


In [13]:
beds

,bed_id,room,adm_id
0,0,0,21
1,1,0,22
2,2,1,7
3,3,1,None
4,4,2,None
5,5,1,19
6,6,3,20
7,7,1,18
8,8,4,25
9,9,2,None


In [14]:
with open('app/data/staff.csv', 'w') as f:
    staffs.to_csv(f, index=False)
with open('app/data/patients.csv', 'w') as f:
    patients.to_csv(f, index=False)
with open('app/data/admissions.csv', 'w') as f:
    admissions.to_csv(f, index=False)
with open('app/data/omr.csv', 'w') as f:
    omr.to_csv(f, index=False)
with open('app/data/prescriptions.csv', 'w') as f:
    prescriptions.to_csv(f, index=False)
with open("app/data/diagnoses.csv", "w") as f:
    diagnoses.to_csv(f, index=False)
with open("app/data/rooms.csv", "w") as f:
    rooms.to_csv(f, index=False)
with open("app/data/beds.csv", "w") as f:
    beds.to_csv(f, index=False)

In [15]:
with open("app/data/admission_types.pkl", "wb") as f:
    pickle.dump(admission_types, f)
with open("app/data/admission_locations.pkl", "wb") as f:
    pickle.dump(admission_locations, f)
with open("app/data/insurance_types.pkl", "wb") as f:
    pickle.dump(insurance_types, f)
with open("app/data/languages.pkl", "wb") as f:
    pickle.dump(languages, f)
with open("app/data/marital_statuses.pkl", "wb") as f:
    pickle.dump(marital_statuses, f)
with open("app/data/races.pkl", "wb") as f:
    pickle.dump(races, f)
with open("app/data/drugs.pkl", "wb") as f:
    pickle.dump(drugs, f)
with open("app/data/diagnoses_types.pkl", "wb") as f:
    pickle.dump(diagnoses_types, f)

In [16]:
patient_db = pd.merge(patients, admissions, how="inner", on="patient_id")
patient_db = pd.merge(patient_db, omr, how="inner", on="adm_id")
patient_db = pd.merge(patient_db, prescriptions, how="inner", on="adm_id")
patient_db = pd.merge(patient_db, diagnoses, how="inner", on="adm_id")

df_helper.drop_columns(
    patient_db,[
        'language',
        'admit_time', 
        'discharge_time',
        'omr_id',
        'prescription_id', 
        'diagnosis_id',
])
patient_db

,patient_id,name,age,gender,insurance,maritalStatus,race,adm_id,type,location,weight,bp_systolic,bp_diastolic,drug,diagnosis
0,0,Tyler Gentry,11,M,Other,SINGLE,BLACK/CAPE VERDEAN,0,SURGICAL SAME DAY ADMISSION,TRANSFER FROM SKILLED NURSING FACILITY,108,114,74,Sodium Chloride,Streptococcus infection in conditions classifi...
1,1,James Hawkins,24,M,Medicare,MARRIED,WHITE - OTHER EUROPEAN,1,ELECTIVE,PROCEDURE SITE,99,172,103,Sodium Chloride,Hypoxemia
2,2,Jessica Cooper,18,F,Medicare,WIDOWED,PORTUGUESE,2,ELECTIVE,EMERGENCY ROOM,116,109,96,Nephrocaps,Attention deficit disorder with hyperactivity
3,3,Yolanda Daniel,63,F,Medicare,MARRIED,WHITE - OTHER EUROPEAN,3,DIRECT EMER.,TRANSFER FROM HOSPITAL,115,116,83,Nephrocaps,"Dysphagia, oropharyngeal phase"
4,4,Ashley Hogan,59,F,Other,DIVORCED,PORTUGUESE,4,URGENT,INTERNAL TRANSFER TO OR FROM PSYCH,93,146,78,Nicotine Polacrilex,"Dysphagia, oropharyngeal phase"
5,5,Ethan Love,62,M,Other,DIVORCED,PATIENT DECLINED TO ANSWER,5,EW EMER.,PACU,94,123,93,Sodium Chloride,Streptococcus infection in conditions classifi...
6,6,Laura Johnson,67,F,Medicaid,DIVORCED,UNKNOWN,6,OBSERVATION ADMIT,PROCEDURE SITE,65,155,85,Ondansetron,"Asthma, unspecified type, unspecified"
7,7,Michael Rosales,67,M,Medicare,SINGLE,HISPANIC OR LATINO,7,ELECTIVE,CLINIC REFERRAL,90,123,119,Nicotine Polacrilex,Methicillin resistant pneumonia due to Staphyl...
8,8,Julia Gomez,77,F,Medicaid,MARRIED,HISPANIC OR LATINO,8,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,96,180,108,Nicotine Polacrilex,"Dysphagia, oropharyngeal phase"
9,9,Patricia Mejia,56,F,Medicaid,DIVORCED,PATIENT DECLINED TO ANSWER,9,AMBULATORY OBSERVATION,TRANSFER FROM HOSPITAL,67,199,101,OLANZapine,"Dysphagia, oropharyngeal phase"


In [17]:
with open("app/data/patient_db.csv", "w") as f:
    patient_db.to_csv(f, index=False)
with open("app/data/hospital_inventory.csv", "w") as f:
    hospital_inventory.to_csv(f, index=False)